A minor issue with the usual manner of using the data algebra
is one has to specify the table twice: once to get the description
of the table and once to cause the execution. This is by design.
Separating the operator specification from use is a major performance
improvement, as it allows the system to know if we are composing
operations or if we are sequencing operations. This also allows re-use
of operator pipelines on related tables, and makes exporting
pipelines to SQL much easier to specify.

Let's look at this issue.

The usual way of using data algebra is as follows

In [1]:
import pandas
from data_algebra.data_ops import *

d = pandas.DataFrame({
  'x': [1, 1, 2],
  'y': [5, 4, 3],
  'z': [6, 7, 8],
})

d

,x,y,z
0,1,5,6
1,1,4,7
2,2,3,8


In [2]:
ops = describe_table(d). \
    drop_columns(['z'])

res_1 = ops.transform(d)

res_1

,x,y
0,1,5
1,1,4
2,2,3


The point being, there may be some dissatisfaction with
having to specify the table twice: once in `describe_table()`,
and once in the transform.

This is by design, but can be avoided by some possible use patterns
For example one can try a container object as follows.

In [3]:
class Immediate():
    def __init__(self, df):
        self.df = df
        self.description = describe_table(self.df)
        self.ops = self.description

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        pass

    def ex(self):
        return context.ops.transform(context.df)

In [4]:
with Immediate(d) as context:
    context.ops = context.description. \
        drop_columns(['z'])
    res_2 = context.ex()

res_2

,x,y
0,1,5
1,1,4
2,2,3


The seemingly unnecessary use of the `with` block is to guarantee the
`Immediate()` object does not stay around holding a reference to the
original table.